# Lab - Classification

In this lab, we are going to build a classification module. When given an image of a handwritten digit like the one below, the model will be able to tell which digit is in the image.

<img src='test2.jpg'>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random


# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

ModuleNotFoundError: No module named 'cv2'

: 

In [ ]:
path_to_dataset = r'digits_dataset'
target_img_size = (32, 32) # fix image size because classification algorithms THAT WE WILL USE HERE expect that

# We are going to fix the random seed to make our experiments reproducible 
# since some algorithms use pseudorandom generators
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

## Part I - Feature Extraction

In this part, we are going to implement three functions. Each one will extract a different set of features from the image. The three sets are:

1. Histogram of the pixel values features (this is the histogram you know, but on the HSV channels)
2. Histogram of Gradients (HoG) features
3. Raw pixels (basically, not doing any feature extraction and just supplying the input image to the classifier)

In [ ]:

def extract_hsv_histogram(img, target_img_size=(128, 128)):
    """
    1. Resize image
    2. Convert BGR → HSV
    3. Compute 3D HSV histogram with bins (8, 8, 8)
    """
    resized_image = cv2.resize(img, target_img_size)
    hsv_img = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist(
        [hsv_img],           
        [0, 1, 2],           
        None,                
        [8, 8, 8],           
        [0, 180, 0, 256, 0, 256]   
    ) 

    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)

    return hist.flatten()
   

In [ ]:
def extract_hog_features(img):
    """
    TODO
    You won't implement anything in this function. You just need to understand it 
    and understand its parameters (i.e win_size, cell_size, ... etc)
    """
    img = cv2.resize(img, target_img_size)
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [ ]:
def extract_raw_pixels(img):
    """
    TODO
    The classification algorithms we are going to use expect the input to be a vector not a matrix. 
    This is because they are general purpose and don't work only on images.
    CNNs, on the other hand, expect matrices since they operate on images and exploit the 
    arrangement of pixels in the 2-D space.
    
    So, what we only need to do in this function is to resize and flatten the image.
    """
    img = cv2.resize(img, target_img_size)
    flattened = img.flatten()
    
    return flattened

In [ ]:
def extract_features(img, feature_set='hog'):
    """
    TODO
    Given either 'hsv_hist', 'hog', 'raw', call the respective function and return its output
    """
    if feature_set == 'hsv_hist':
        return extract_hsv_histogram(img)
    
    elif feature_set == 'hog':
        return extract_hog_features(img)
    
    elif feature_set == 'raw':
        return extract_raw_pixels(img)
    
    else:
        raise ValueError("Invalid feature_set. Choose from: 'hsv_hist', 'hog', 'raw'")

The following function will extract the features and the label of each image in our dataset and save it in RAM. We normally don't save datasets in RAM, but this dataset is small.

In [ ]:
def load_dataset(feature_set='hog'):
    features = []
    labels = []
    img_filenames = os.listdir(path_to_dataset)

    for i, fn in enumerate(img_filenames):
        if fn.split('.')[-1] != 'jpg':
            continue

        label = fn.split('.')[0]
        labels.append(label)

        path = os.path.join(path_to_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_features(img, feature_set))
        
        # show an update every 1,000 images
        if i > 0 and i % 1000 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames)))
        
    return features, labels        

## Part II - Classification

In this part, we will test the classification performance of SVM, KNN, & NNs given our features.

In [ ]:
# TODO understand the hyperparameters of each classifier
classifiers = {
    'SVM': svm.LinearSVC(random_state=random_seed),
    'KNN': KNeighborsClassifier(n_neighbors=7),
    'NN': MLPClassifier(solver='sgd', random_state=random_seed, hidden_layer_sizes=(500,), max_iter=20, verbose=1)
}

In [ ]:
# This function will test all our classifiers on a specific feature set
def run_experiment(feature_set):
    
    # Load dataset with extracted features
    print('Loading dataset. This will take time ...')
    features, labels = load_dataset(feature_set)
    print('Finished loading dataset.')
    
    # Since we don't want to know the performance of our classifier on images it has seen before
    # we are going to withhold some images that we will test the classifier on after training 
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    
    for model_name, model in classifiers.items():
        print('############## Training', model_name, "##############")
        # Train the model only on the training features
        model.fit(train_features, train_labels)
        
        # Test the model on images it hasn't seen before
        accuracy = model.score(test_features, test_labels)
        
        print(model_name, 'accuracy:', accuracy*100, '%')

Now, we see how each classifier and each feature set performs

In [ ]:
run_experiment('hog')
"""
You should get the following test accuracies the first time 

SVM accuracy ~ 97.70833333333333
KNN accuracy ~ 96.52777777777779
NN accuracy ~ 93.95833333333333
"""

In [ ]:
run_experiment('hsv_hist')
"""
You should get the following test accuracies the first time 

SVM accuracy ~ 32.083333333333336
KNN accuracy ~ 32.708333333333336
NN accuracy ~ 9.722222222222223
"""

# Why low accuracies?

In [ ]:
run_experiment('raw')
"""
You should get the following test accuracies the first time 

SVM accuracy ~ 85.06944444444444
KNN accuracy ~ 93.95833333333333
NN accuracy ~ 88.68055555555556
"""

The classifiers list now has models trained on the last feature set you ran an experiment on. You can play around with it checking the probability it gives to each label, given an image.

In [ ]:
# Example
test_img_path = r'test2.jpg'
img = cv2.imread(test_img_path)
features = extract_features(img, 'raw')  # be careful of the choice of feature set

In [ ]:
nn = classifiers['NN']
nn.predict_proba([features])

Try to get a better accuracy by changing the model hyperparameters and retraining.